# 311 Calls per NC

Simple idioms I use to display with ipyleaflet.

In this notebook we'll use one of the attributes of the 311 data (nc) to focus on the map.

Steps:

1.  Initial imports
2.  Read data
3.  Explore the data
4.  Pick an nc and build map layers
5.  Render on the map

# 1. Setup

I'm assuming you have an environment ala the environment.yml file.  

I have a startup script in the ~/.ipython/profile_default/startup directory.  I've copied it to this folder so you can run it for the same effect.

**Note:** I'm importing the utility function read_new311_shape to read the preproccessed 311 data and map the column names/types.

In [ ]:
%run start.py
import nc

from utils import read_new311_shape

# 2. Get the Data

In [ ]:
%%time
new311_gdf = read_new311_shape('../data/311/clean311.geojson.zip')

In [ ]:
ncs_gdf = gpd.read_file('../data/neighborhoods/Neighborhood_Councils_(Certified)_cleaned.shp')

In [ ]:
ncs_gdf.rename(columns={'NAME': 'name',
                        'NC_ID': 'nc_id',
                        'SERVICE_RE': 'service_region'},
              inplace=True);

# 3. Examine the Data

In [ ]:
new311_gdf.columns

In [ ]:
new311_gdf.info()

In [ ]:
new311_gdf.request_type.value_counts()

In [ ]:
new311_gdf.nc_name.value_counts().to_frame().head().rename(columns={'nc_name': 'count'})

In [ ]:
new311_gdf.columns

In [ ]:
new311_gdf.head()

# 4 - Select NC and Build Map Layers

After some digging I'm going with Little Tokyo (nc == 46).  Not a lot of points so the map will perform.

(12/29/2021) - Not sure yet if ipyleaflet is the way to go?  

In [ ]:
small_gdf = new311_gdf.query(f"nc == 46", engine='python').reset_index().drop(columns=['index'])
#small_gdf = new311_gdf.query(f"nc == 79").reset_index().drop(columns=['index'])

In [ ]:
small_gdf.request_type.value_counts()

In keeping with our basic theme, it's predominately Graffiti Removal (FWTW).

In [ ]:
small_gdf.created_dt.min()

In [ ]:
small_gdf.created_dt.max()

In [ ]:
(_ - __).days / 7

You can see this is from a dataset pull in early December.  It's 47 weeks worth of requests.

When I get to the timeline viz that will help.

Pretty simple hack to build a color map for request types.  I found these colors on H4LA github so maybe they'll be close to colors on the 311-data.org?

In [ ]:
keys = new311_gdf.request_type.value_counts().keys().to_list()

vals = [
    '#267370',
    '#8B508B',
    '#EDAD08',
    '#1D6996',
    '#11975F',
    '#E17C05',
    '#685DB1',
    '#AE3D51',
    '#79B74E',
    '#BF82BA',
    '#D05F4E',
    '#33BBEE'
]

c_map = {kv[0]: kv[1] for kv in zip(keys, vals)}

c_map

This is one of my very first idioms for markers and ipyleaflet maps.  I should rework this, but later.

Note: I'm using the MarkerCluster widget.  It's somewhat better with large volume of markers and scrolling on the map.

In [ ]:
markers = list()

for i, row in tqdm(small_gdf.iterrows()):
    
    fill_color = c_map[row.request_type]
    marker = CircleMarker(location=(row.geometry.y, row.geometry.x), radius=5, stroke=False, fill_color=fill_color, fill_opacity=1.0)
    msg = HTML()
    msg.value = "report: {}<br>Address: {}</br>when: {}<br/>type: {}".format(row['SRNumber'], 
                                                              row['address'],
                                                            row['created_dt'].strftime("%m/%d/%Y, %H:%M"),
                                                            row['request_type']) #"status: {}<br/>coord: {}".format(r['status'], r['coordinates'])
    marker.popup = msg
    markers.append(marker)
    small_gdf.loc[i, 'marker'] = marker

little_tokyo_311_cluster = MarkerCluster(markers=markers, name='311 Calls - Clustered')


# 5 - Display the Map

The last step in our map display adventure.

I will add 3 layers:

1. Little Tokyo NC polygon
2. The MarkerCluster markers
3. All the markers.

You can select/deselect the layers to see.

Note: I have 4 base layers too.  Give some flexiblity on the map/viz.

Find the map center from the neighborhood council polygon.

In [ ]:
#ncs_gdf.to_crs(epsg=4326, inplace=True)
little_tokyo_gdf = ncs_gdf.query(f"nc_id == 46", engine='python').reset_index()

center = little_tokyo_gdf.iloc[0].geometry.centroid.y, little_tokyo_gdf.iloc[0].geometry.centroid.x

In [ ]:
nc_layer = GeoData(geo_dataframe = little_tokyo_gdf,
                   style={'color': 'black', 'fillColor': '#3366cc', 'opacity':0.8, 'weight':1.9, 'dashArray':'5', 'fillOpacity':0.2},
                   hover_style={'fillColor': 'red' , 'fillOpacity': 0.2},
                   name = 'Little Tokyo NC')

In [ ]:
imagery = basemap_to_tiles(basemaps.Esri.WorldImagery)
imagery.base = True
osm = basemap_to_tiles(basemaps.OpenStreetMap.Mapnik)
osm.base = True

from ipyleaflet import TileLayer
google_map = TileLayer(
    url="https://mt1.google.com/vt/lyrs=m&x={x}&y={y}&z={z}",
    attribution="Google",
    name="Google Maps",
)
google_map.base = True

google_satellite = TileLayer(
    url="https://mt1.google.com/vt/lyrs=y&x={x}&y={y}&z={z}",
    attribution="Google",
    name="Google Satellite"
)
google_satellite.base = True

map_display = Map(center=center, zoom=15,
                  layers=[google_satellite, google_map, imagery, osm],
                  layout=Layout(height="700px"),
                  scroll_wheel_zoom=True)

map_display.add_control(LayersControl())

#map_display.add_layer(geo_data)
map_display += nc_layer
map_display.add_layer(little_tokyo_311_cluster)

all_311 = LayerGroup(name=f"All 311", layers=markers)
map_display += all_311
map_display

So that is all for now.  I really want to add a timeslider with brushes for crossfiltering.

After much research I may have found the best way to do it, but that is for another day!